In [1]:
import spacy
from typing import Tuple
import nltk
import numpy as np
import pandas as pd
from functools import partial
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from scipy.stats import pearsonr

from zp_ihlt_project.config import TRAIN_DATA_DIR, TEST_DATA_DIR

In [2]:
all_train_dt = pd.read_csv("../data/processed/train_data_with_features.csv")
all_test_dt = pd.read_csv("../data/processed/test_data_with_features.csv")
feature_names = [col for col in all_train_dt.columns if col.startswith("score_")]

In [3]:
selected_features = feature_names

In [4]:
train_datasets = all_train_dt.dataset.unique().tolist()
test_datasets = all_test_dt.dataset.unique().tolist()

In [6]:
dt = all_train_dt

X_train, X_val, y_train, y_val = train_test_split(dt[selected_features], dt.gs, test_size=0.2, random_state=42)
X_test = all_test_dt[selected_features]
y_test = all_test_dt.gs

results = []

# Define parameter grid
param_grid = {
    'n_estimators': [100],
    'max_depth': [3, 6],
    'learning_rate': [0.01, 0.1],
}

model = xgb.XGBRegressor()

# Perform grid search with 5-fold cross validation
grid_search = GridSearchCV(
    model, 
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

# Fit grid search
grid_search.fit(X_train, y_train)

# Print best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best RMSE:", np.sqrt(-grid_search.best_score_))

best_model = grid_search.best_estimator_
preds = best_model.predict(X_train)
results.append(pearsonr(y_train, preds)[0])

preds = best_model.predict(X_val)
results.append(pearsonr(y_val, preds)[0])

preds = best_model.predict(X_test)
results.append(pearsonr(y_test, preds)[0])

results = pd.DataFrame([results], columns=["train_pearson", "val_pearson", "test_pearson"], index=["all"])
results

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
Best RMSE: 0.7437144351720009


train_pearson  val_pearson  test_pearson
all       0.935893     0.834333      0.718027

In [7]:
best_model.fit(dt[selected_features], dt.gs)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [8]:
train_results = []
datasets = train_datasets
for dataset in datasets:
    dataset_results = []
    dataset_results.append(dataset)
    dt = all_train_dt[all_train_dt.dataset == dataset]
    test_dt = all_test_dt[all_test_dt.dataset == dataset]

    X_train, X_val, y_train, y_val = train_test_split(dt[selected_features], dt.gs, test_size=0.2, random_state=42)
    X_test = test_dt[selected_features]
    y_test = test_dt.gs

    preds = best_model.predict(X_train)
    dataset_results.append(pearsonr(y_train, preds)[0])

    preds = best_model.predict(X_val)
    dataset_results.append(pearsonr(y_val, preds)[0])

    preds = best_model.predict(X_test[selected_features])
    dataset_results.append(pearsonr(y_test, preds)[0])

    train_results.append(dataset_results)

train_results = pd.DataFrame(train_results, columns=["dataset", "train_pearson", "val_pearson", "test_pearson"])
train_results

dataset  train_pearson  val_pearson  test_pearson
0       MSRpar       0.831700     0.838021      0.644561
1       MSRvid       0.904660     0.934363      0.833917
2  SMTeuroparl       0.824211     0.827384      0.489484

In [9]:
test_results = []
datasets = test_datasets
for dataset in datasets:
    dataset_results = []
    dataset_results.append(dataset)
    test_dt = all_test_dt[all_test_dt.dataset == dataset]

    X_test = test_dt[selected_features]
    y_test = test_dt.gs

    preds = best_model.predict(X_test[selected_features])
    dataset_results.append(pearsonr(y_test, preds)[0])

    test_results.append(dataset_results)

preds = best_model.predict(all_test_dt[selected_features])
test_results.append(['all', pearsonr(all_test_dt.gs, preds)[0]])
test_results = pd.DataFrame(test_results, columns=["dataset", "test_pearson"])
test_results

dataset  test_pearson
0       MSRpar      0.644561
1       MSRvid      0.833917
2  SMTeuroparl      0.489484
3         OnWN      0.662742
4      SMTnews      0.432068
5          all      0.722297

In [9]:
results_to_beat = pd.DataFrame(np.array([[.683, .873, .528, .664, .493, 0.823]]).T, index=[*test_datasets, 'all'], columns=["pearson_to_beat"])
results_to_beat

pearson_to_beat
MSRpar                 0.683
MSRvid                 0.873
SMTeuroparl            0.528
OnWN                   0.664
SMTnews                0.493
all                    0.823